## Testing fragments from different editions

In [92]:
import pandas as pd
import numpy as np

import re

from sklearn.preprocessing import StandardScaler

import logging

logging.basicConfig(level="INFO")
logger = logging.getLogger("ruzicka")

from ruzicka.BDIVerifier import BDIVerifier

In [119]:
# load preprocessed data

corpus = pd.read_csv('04_tests/ed1780_2k_200mfw_rfreq.csv') # rel freq

corpus

,work,author,chunk_num,tag,de,des,et,la,les,vous,...,voit,rendre,beaucoup,seul,ici,mieux,donner,enfin,voir,cependant
0,Avis,Baudeau,0,0__Baudeau_Avis,3.45,2.20,2.50,1.40,2.35,0.65,...,0.05,0.00,0.05,0.05,0.05,0.00,0.05,0.20,0.00,0.00
1,Eclaircissemens,Baudeau,0,0__Baudeau_Eclaircissemens,4.40,2.85,2.80,2.60,2.40,1.65,...,0.05,0.00,0.00,0.05,0.00,0.00,0.05,0.00,0.00,0.00
2,Explication,Baudeau,0,0__Baudeau_Explication,4.90,2.60,3.80,3.10,4.65,0.95,...,0.00,0.00,0.05,0.10,0.10,0.00,0.05,0.05,0.05,0.00
3,Idees sur l-administration,Baudeau,0,0__Baudeau_Idees sur l-administration,5.60,1.90,2.20,3.65,1.85,0.10,...,0.00,0.00,0.10,0.10,0.00,0.05,0.05,0.10,0.00,0.05
4,Idees sur la puissance,Baudeau,0,0__Baudeau_Idees sur la puissance,4.90,1.85,2.50,2.35,2.75,0.10,...,0.00,0.10,0.05,0.00,0.00,0.10,0.00,0.10,0.05,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4469,elements T3,Marmontel,96,96__Marmontel_elements T3,4.15,1.20,3.35,3.00,1.15,0.00,...,0.05,0.10,0.00,0.25,0.00,0.05,0.05,0.05,0.00,0.00
4470,Histoire Generale T19,Deleyre,97,97__Deleyre_Histoire Generale T19,5.25,1.85,0.75,3.75,2.65,0.00,...,0.00,0.05,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4471,elements T3,Marmontel,97,97__Marmontel_elements T3,4.60,0.95,3.10,3.40,1.55,0.00,...,0.00,0.10,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00
4472,Histoire Generale T19,Deleyre,98,98__Deleyre_Histoire Generale T19,4.90,2.95,0.75,3.25,3.20,0.05,...,0.05,0.00,0.20,0.00,0.00,0.05,0.00,0.00,0.05,0.05


In [120]:
raw_df = corpus.iloc[:, :4] # leaves metadata
X = corpus.iloc[:, 4:] # leaves only word columns

raw_df

,work,author,chunk_num,tag
0,Avis,Baudeau,0,0__Baudeau_Avis
1,Eclaircissemens,Baudeau,0,0__Baudeau_Eclaircissemens
2,Explication,Baudeau,0,0__Baudeau_Explication
3,Idees sur l-administration,Baudeau,0,0__Baudeau_Idees sur l-administration
4,Idees sur la puissance,Baudeau,0,0__Baudeau_Idees sur la puissance
...,...,...,...,...
4469,elements T3,Marmontel,96,96__Marmontel_elements T3
4470,Histoire Generale T19,Deleyre,97,97__Deleyre_Histoire Generale T19
4471,elements T3,Marmontel,97,97__Marmontel_elements T3
4472,Histoire Generale T19,Deleyre,98,98__Deleyre_Histoire Generale T19


In [121]:
# numer of rows and columns in the X (word frequencies subset)
X.shape

(4474, 200)

In [122]:
# create numeric author labels
labels, label_uniques = raw_df.author.factorize()
raw_df.insert(1, "author_label", labels)
raw_df

,work,author_label,author,chunk_num,tag
0,Avis,0,Baudeau,0,0__Baudeau_Avis
1,Eclaircissemens,0,Baudeau,0,0__Baudeau_Eclaircissemens
2,Explication,0,Baudeau,0,0__Baudeau_Explication
3,Idees sur l-administration,0,Baudeau,0,0__Baudeau_Idees sur l-administration
4,Idees sur la puissance,0,Baudeau,0,0__Baudeau_Idees sur la puissance
...,...,...,...,...,...
4469,elements T3,11,Marmontel,96,96__Marmontel_elements T3
4470,Histoire Generale T19,3,Deleyre,97,97__Deleyre_Histoire Generale T19
4471,elements T3,11,Marmontel,97,97__Marmontel_elements T3
4472,Histoire Generale T19,3,Deleyre,98,98__Deleyre_Histoire Generale T19


### NB CHANGE THE LABEL

In [123]:
# select an "unknown" work
# ed1770-CH1774-CH1780
# ed1774-CH1780

problems = raw_df[raw_df.work == "ed1780"].reset_index(drop=True).copy()
problems_X = X[raw_df.work == "ed1780"].reset_index(drop=True).copy()
problems

,work,author_label,author,chunk_num,tag
0,ed1780,7,HDI,0,0__HDI_ed1780
1,ed1780,7,HDI,1,1__HDI_ed1780
2,ed1780,7,HDI,10,10__HDI_ed1780
3,ed1780,7,HDI,11,11__HDI_ed1780
4,ed1780,7,HDI,12,12__HDI_ed1780
5,ed1780,7,HDI,13,13__HDI_ed1780
6,ed1780,7,HDI,14,14__HDI_ed1780
7,ed1780,7,HDI,15,15__HDI_ed1780
8,ed1780,7,HDI,16,16__HDI_ed1780
9,ed1780,7,HDI,17,17__HDI_ed1780


In [124]:
# extract the rest of the corpus

rest = raw_df[raw_df.work != "ed1780"].reset_index(drop=True).copy()
rest_X = X[raw_df.work != "ed1780"].reset_index(drop = True).copy()
rest

,work,author_label,author,chunk_num,tag
0,Avis,0,Baudeau,0,0__Baudeau_Avis
1,Eclaircissemens,0,Baudeau,0,0__Baudeau_Eclaircissemens
2,Explication,0,Baudeau,0,0__Baudeau_Explication
3,Idees sur l-administration,0,Baudeau,0,0__Baudeau_Idees sur l-administration
4,Idees sur la puissance,0,Baudeau,0,0__Baudeau_Idees sur la puissance
...,...,...,...,...,...
4426,elements T3,11,Marmontel,96,96__Marmontel_elements T3
4427,Histoire Generale T19,3,Deleyre,97,97__Deleyre_Histoire Generale T19
4428,elements T3,11,Marmontel,97,97__Marmontel_elements T3
4429,Histoire Generale T19,3,Deleyre,98,98__Deleyre_Histoire Generale T19


**Scaling**

In [125]:
sts = StandardScaler(with_mean=False).fit(rest_X)
rest_scaled_X = sts.transform(rest_X)
problems_scaled_X = sts.transform(problems_X)

**Verification**

In [126]:
# check unique authors
label_uniques.values

array(['Baudeau', 'Chastellux', 'Condorcet', 'Deleyre', 'dHolbach',
       'Diderot', 'Guibert', 'HDI', 'Jaucourt', 'Jussieu', 'La Grange',
       'Marmontel', 'Meister', 'Morellet', 'Naigeon', 'Pechmeja',
       'Raynal', 'Rivière', 'Saint-Lambert'], dtype=object)

In [127]:
# set verifier
rng = np.random.default_rng(42)

bdi_mm = BDIVerifier(
    metric='minmax', nb_bootstrap_iter=1000, rnd_prop=0.35, random_state=rng
)

In [128]:
# fit 
bdi_mm.fit(rest_scaled_X, rest.author_label)

01/22/2025 04:09:33 [ruzicka:INFO] Fitting on 4431 documents...


In [129]:
for label in label_uniques.values:
    print(f"Testing against {label}")
    code = label_uniques.get_loc(label)
    print(
        f"Bootstrap Match Strength (one per chunk, 0-1.0): {bdi_mm.predict_proba(problems_scaled_X, [code] * problems_scaled_X.shape[0])}"
    )

01/22/2025 04:09:39 [ruzicka:INFO] Predicting on 43 documents


Testing against Baudeau


01/22/2025 04:09:48 [ruzicka:INFO] Predicting on 43 documents


Bootstrap Match Strength (one per chunk, 0-1.0): [0.736 0.896 0.686 0.712 0.538 0.878 0.606 0.787 0.902 0.67  0.827 0.864
 0.633 0.613 0.642 0.816 0.894 0.564 0.694 0.792 0.88  0.865 0.844 0.622
 0.805 0.373 0.484 0.664 0.933 0.92  0.668 0.857 0.851 0.847 0.685 0.713
 0.744 0.534 0.858 0.611 0.838 0.67  0.863]
Testing against Chastellux


01/22/2025 04:09:58 [ruzicka:INFO] Predicting on 43 documents


...

Bootstrap Match Strength (one per chunk, 0-1.0): [0.368 0.633 0.671 0.648 0.505 0.759 0.327 0.333 0.473 0.508 0.263 0.163
 0.576 0.276 0.289 0.288 0.293 0.326 0.181 0.53  0.551 0.412 0.465 0.747
 0.419 0.401 0.28  0.224 0.415 0.223 0.503 0.38  0.236 0.406 0.584 0.189
 0.607 0.477 0.215 0.305 0.279 0.18  0.428]
Testing against Raynal


01/22/2025 04:13:07 [ruzicka:INFO] Predicting on 43 documents


Bootstrap Match Strength (one per chunk, 0-1.0): [0.946 0.414 0.628 0.953 0.728 0.843 0.962 0.749 0.585 0.813 0.694 0.717
 0.829 0.791 0.838 0.851 0.848 0.507 0.535 0.764 0.646 0.627 0.966 0.821
 0.708 0.941 0.95  0.928 0.966 0.739 0.949 0.983 0.834 0.877 0.8   0.485
 0.822 0.826 0.834 0.961 0.861 0.885 0.809]
Testing against Rivière


01/22/2025 04:13:15 [ruzicka:INFO] Predicting on 43 documents


Bootstrap Match Strength (one per chunk, 0-1.0): [0.349 0.838 0.619 0.344 0.559 0.618 0.511 0.159 0.619 0.706 0.778 0.669
 0.31  0.405 0.588 0.542 0.518 0.866 0.91  0.823 0.806 0.932 0.669 0.283
 0.739 0.28  0.474 0.478 0.216 0.511 0.366 0.404 0.697 0.734 0.772 0.552
 0.676 0.609 0.431 0.461 0.295 0.625 0.593]
Testing against Saint-Lambert
Bootstrap Match Strength (one per chunk, 0-1.0): [0.835 0.896 0.967 0.933 0.98  0.885 0.934 0.895 0.897 0.83  0.863 0.818
 0.912 0.751 0.867 0.783 0.748 0.855 0.876 0.973 0.93  0.924 0.944 0.916
 0.655 0.851 0.868 0.939 0.777 0.753 0.952 0.917 0.845 0.905 0.861 0.767
 0.803 0.945 0.693 0.848 0.926 0.879 0.884]


### NB CHANGE PATH FOR NEW TESTS

In [130]:
authors = label_uniques.values

fh = ''

for a in authors:

    print(a)
    
    bdi_mm.predict_proba(
        problems_scaled_X, [label_uniques.get_loc(a)] * problems_scaled_X.shape[0]
    )

    
    x = pd.DataFrame(dict(zip(problems.tag, bdi_mm._dist_arrays)))
    
    fh = '04_tests/tests/ed1780_vs_' + a + '.csv'

    x.to_csv(fh)

01/22/2025 04:13:36 [ruzicka:INFO] Predicting on 43 documents


Baudeau


01/22/2025 04:13:45 [ruzicka:INFO] Predicting on 43 documents


Chastellux


01/22/2025 04:13:55 [ruzicka:INFO] Predicting on 43 documents


...

Raynal


01/22/2025 04:17:06 [ruzicka:INFO] Predicting on 43 documents


Rivière


01/22/2025 04:17:14 [ruzicka:INFO] Predicting on 43 documents


Saint-Lambert
